In [1]:
import numpy as np
from os.path import normpath as fn
import autograd 
import layers

np.random.seed(0)

In [2]:
# Load data
data = np.load(fn('data/mnist_26k.npz'))

train_im = np.float32(data['im_train'])/255.-0.5
train_lb = data['lbl_train']

val_im = np.float32(data['im_val'])/255.-0.5
val_lb = data['lbl_val']

In [3]:
###################################
# build static computational graph
###################################
nHidden = 2048

# placeholder for input
inp = autograd.Value()
lab = autograd.Value()

model = layers.Sequential([layers.Linear(28*28, nHidden),
                           layers.RELU(),
                           layers.Linear(nHidden, 10)
                          ])
print(model)

# loss function: softmax + crossentropy
smaxloss = layers.SmaxCELoss()
accuracy = layers.Accuracy()

# build computational graph
y = model(inp)

# Cross Entropy of Soft-max
loss = smaxloss(y,lab)

# Accuracy
acc = accuracy(y,lab)

Sequential(
  (0) Linear(784, 2048)
  (1) RELU()
  (2) Linear(2048, 10)
)


In [5]:
# Training loop
BSZ=64
lr=0.001

NUM_EPOCH=10
DISPITER=100
batches = range(0,len(train_lb)-BSZ+1,BSZ)

autograd.init_momentum()


niter, avg_loss, avg_acc=0, 0., 0.
for ep in range(NUM_EPOCH+1):
    # As we train, let's keep track of val accuracy
    vacc, vloss, viter= 0., 0., 0
    for b in range(0,len(val_lb)-BSZ+1,BSZ):
        inp.set(val_im[b:b+BSZ,:])
        lab.set(val_lb[b:b+BSZ])
        autograd.Forward()
        viter += 1
        vacc += acc.top
        vloss += loss.top
    vloss, vacc = vloss / viter, vacc / viter * 100
    print("%09d: #### %d Epochs: Val Loss = %.3e, Accuracy = %.2f%%" % (niter,ep,vloss,vacc))
    if ep == NUM_EPOCH:
        break

    # Shuffle Training Set
    idx = np.random.permutation(len(train_lb))

    # Train one epoch
    for b in batches:
        # Load a batch
        inp.set(train_im[idx[b:b+BSZ],:])
        lab.set(train_lb[idx[b:b+BSZ]])

        autograd.Forward()
        avg_loss += loss.top 
        avg_acc += acc.top
        niter += 1
        if niter % DISPITER == 0:
            avg_loss = avg_loss / DISPITER
            avg_acc = avg_acc / DISPITER * 100
            print("%09d: Training Loss = %.3e, Accuracy = %.2f%%" % (niter,avg_loss,avg_acc))
            avg_loss, avg_acc = 0., 0.

        autograd.Backward(loss)
        autograd.momentum(lr,0.9)

000000000: #### 0 Epochs: Val Loss = 5.271e-01, Accuracy = 88.44%
000000100: Training Loss = 5.530e-01, Accuracy = 86.44%
000000200: Training Loss = 5.063e-01, Accuracy = 87.27%
000000300: Training Loss = 4.849e-01, Accuracy = 87.61%
000000390: #### 1 Epochs: Val Loss = 3.998e-01, Accuracy = 90.10%
000000400: Training Loss = 4.525e-01, Accuracy = 88.19%
000000500: Training Loss = 4.380e-01, Accuracy = 88.48%
000000600: Training Loss = 4.287e-01, Accuracy = 88.47%
000000700: Training Loss = 4.206e-01, Accuracy = 88.30%
000000780: #### 2 Epochs: Val Loss = 3.502e-01, Accuracy = 91.56%
000000800: Training Loss = 3.963e-01, Accuracy = 89.19%
000000900: Training Loss = 3.858e-01, Accuracy = 89.61%
000001000: Training Loss = 3.750e-01, Accuracy = 89.53%
000001100: Training Loss = 3.839e-01, Accuracy = 89.25%
000001170: #### 3 Epochs: Val Loss = 3.268e-01, Accuracy = 91.77%
000001200: Training Loss = 3.721e-01, Accuracy = 89.84%
000001300: Training Loss = 3.560e-01, Accuracy = 89.97%
00000140